### Домашняя работа к уроку 3

In [86]:
import numpy as np
import matplotlib.pyplot as plt

In [87]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [88]:
X.shape

(10, 4)

In [89]:
y.shape

(10,)

In [90]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [91]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [92]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

#### Задача 1
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [93]:
# Исходный вариант кода:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

Т.к. log(0) не определен, а при x->0+ log(x)=-inf, то нам необходимо
избежать попадание 0 под логарифм. 
Т.о. функция calc_logloss модифицируется:

In [94]:
def calc_logloss_modif(y, y_pred):
    for i in range(len(y)):
        if y_pred[i] == 0:
            y_pred[i]=y_pred[i] + (1e-8)
        elif y_pred[i] == 1:
            y_pred[i]=y_pred[i] - (1e-8)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [95]:
# Тестируем функцию calc_logloss_modif критическими случаями:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss_modif(y1, y_pred1)

0.052680262828913194

In [96]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss_modif(y1, y_pred1)

0.052680262828913194

In [97]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss_modif(y1, y_pred1)

0.10536051565782628

#### Задача 2
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным

In [98]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [101]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss_modif(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        #if i % (iterations / 10) == 0:
        #    print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return err

In [103]:
best_error = np.inf
iterations_best = 0
eta_best = 0
for iterations in [10, 100, 1000, 10000]:
    for eta in [1e-8, 1e-6, 1e-4, 1e-2, 1, 2, 3, 4, 6, 8, 10, 12, 14, 16]:
        error = eval_model(X_st, y, iterations=iterations, eta=eta)
        #print(f'{error=}, {iterations=}, {eta=}')
        if error < best_error:
            best_error = error
            iterations_best = iterations
            eta_best = eta
print(f'{best_error=}, {iterations_best=}, {eta_best=}')

best_error=0.023009894586955022, iterations_best=10000, eta_best=16


#### Задача 3
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba

In [118]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(X @ W)
    return y_pred_proba

In [119]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W

In [120]:
W = eval_model(X_st, y, 10000, 1e-4)

0 [ 0.49667621 -0.13840939  0.6476858   1.52297324] 1.1785958344356262
1000 [ 0.46070853 -0.27660658  0.64395225  1.46886563] 0.9450063518188168
2000 [ 0.42910538 -0.39762907  0.63899133  1.42101856] 0.7649678531356925
3000 [ 0.40237714 -0.49496601  0.63497974  1.38181554] 0.6467938123731767
4000 [ 0.38047894 -0.5650296   0.63387164  1.35257212] 0.5836344091291391
5000 [ 0.36247418 -0.61238439  0.63568618  1.33181421] 0.5532531003505643
6000 [ 0.34723076 -0.64431226  0.63950668  1.31708294] 0.5382593304625718
7000 [ 0.33391048 -0.6662408   0.64455548  1.30646938] 0.5302255883714215
8000 [ 0.32196306 -0.68156712  0.65034185  1.29871977] 0.5254818665326442
9000 [ 0.3110257  -0.69240217  0.65656725  1.29301615] 0.5223831652167139


In [121]:
calc_pred_proba(W, X_st)

array([0.55973106, 0.62563756, 0.76325904, 0.08139218, 0.80654274,
       0.79419209, 0.79974273, 0.19277038, 0.54282025, 0.81942788])

#### Задача 4
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred)

In [128]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X_st)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [131]:
pred = calc_pred(W, X_st)
pred

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1])

#### Задача 5
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score

In [132]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [142]:
y_pred = calc_pred(W, X_st)
y_pred

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1])

*Accuracy*

In [140]:
def accuracy(y_pred, y):
    s = 0
    for i in range(len(y)):
        if y[i]==y_pred[i]:
            s+=1
    return s/len(y)

In [141]:
print(accuracy(y_pred, y))

0.7


*Confusion Matrix, Precission, Recall, F-score*
##### | TP       FP |
##### | FN      TN |

In [164]:
def conf_matrix(y_pred, y, beta=1):
    cm = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: # TP - True Positive
            cm[0][0] += 1
        elif y[i] == y_pred[i] == 0: # TN - True Negative
            cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: # FN - False Negative
            cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: # FP - False Positive
            cm[0][1] += 1
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    precission = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = (1 + beta**2) * precission * recall / ((beta**2)*precission + recall)
    
    print(f'Confusion matrix:\n{cm}')
    print(f'{precission=},\n{recall=},\n{f_score=}')
    
    return cm

In [165]:
conf_matrix(y_pred, y)

Confusion matrix:
[[5. 3.]
 [0. 2.]]
precission=0.625,
recall=1.0,
f_score=0.7692307692307693


array([[5., 3.],
       [0., 2.]])